In [ ]:
# Mount google drive
from google.colab import drive
!mkdir /gdrive
drive.mount('/gdrive')

In [ ]:
import os

In [ ]:
import cv2, numpy as np, pathlib, concurrent.futures as cf

In [ ]:
orange_temp = os.path.join('/gdrive/MyDrive/object_detection_company', 'orange_temp.jpg')
apple_temp = os.path.join('/gdrive/MyDrive/object_detection_company', 'apple_temp.jpg')

input_dir = os.path.join('/gdrive/MyDrive/object_detection_company', 'input')
output_dir = os.path.join('/gdrive/MyDrive/object_detection_company', 'output')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.imread(orange_temp))
cv2_imshow(cv2.imread(apple_temp))

In [ ]:
from pathlib import Path
def frames_from_camera():
    for img_path in Path(input_dir).glob("*.jpg"):
        yield cv2.imread(str(img_path))

In [ ]:
for img_path in Path(input_dir).glob("*.jpg"):
  print(str(img_path))

In [ ]:
# --- ① テンプレート準備（起動時に一度だけ） ---
tmpl_paths = [orange_temp, apple_temp]
#orb = cv2.ORB_create(nfeatures=1200)
sift = cv2.SIFT_create()
tmpl_db = []     # [(kp, des, corners4, label), ...]

for p in tmpl_paths:
    img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
    #kp, des = orb.detectAndCompute(img, None)
    kp, des = sift.detectAndCompute(img, None)
    h, w = img.shape
    corners = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
    tmpl_db.append((kp, des, corners, pathlib.Path(p).stem))

#bf = cv2.BFMatcher(cv2.NORM_HAMMING)
# SIFT 用 BFMatcher（float型）には NORM_L2 を使う
bf = cv2.BFMatcher(cv2.NORM_L2)

In [ ]:
# --- ② 検出ループ ---
def detect_and_crop(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = sift.detectAndCompute(gray, None)
    if des_frame is None: return []

    crops = []
    for kp_tmpl, des_tmpl, corners, label in tmpl_db:
        matches = bf.knnMatch(des_tmpl, des_frame, k=2)
        good = [m for m,n in matches if m.distance < 0.95*n.distance]

        if len(good) < 10:             # inlier閾値調整
            continue

        src_pts = np.float32([kp_tmpl[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        if len(src_pts) < 4:
            continue

        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 3.0)
        if H is None or mask is None or mask.sum() < 10:
            continue

        inliers = mask.sum()
        if inliers < 5: continue

        proj = cv2.perspectiveTransform(corners, H)
        x, y, w, h = cv2.boundingRect(proj)
        pad = int(0.05 * max(w, h))
        crop = frame_bgr[max(0, y - pad):y + h + pad, max(0, x - pad):x + w + pad]

        crops.append((crop, label, inliers))
    return crops

# --- ③ 並列保存例 ---
executor = cf.ThreadPoolExecutor()

for frame_id, frame in enumerate(frames_from_camera()):
    crops = detect_and_crop(frame)
    print(f"frame {frame_id}: {len(crops)} objects found")  # ← 追加してみて
    for i,(crop,label,score) in enumerate(detect_and_crop(frame)):
        print(i)
        executor.submit(cv2.imwrite,
                        os.path.join(output_dir, f"{label}_{frame_id}_{i}_{score}.png"),
                        crop)


In [ ]:
# --- ② 画像に特徴点を描画する関数 ---
def detect_and_annotate(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = sift.detectAndCompute(gray, None)
    if des_frame is None:
        return []                                      # 何も描けない

    annotated_imgs = []                                # [(annotated_img, label, inliers), …]

    # テンプレートごとにマッチング
    for kp_tmpl, des_tmpl, tmpl_corners, label in tmpl_db:
        matches = bf.knnMatch(des_tmpl, des_frame, k=2)
        good = [m for m, n in matches if m.distance < 0.95 * n.distance]
        if len(good) < 10:
            continue

        src_pts = np.float32([kp_tmpl[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        if len(src_pts) < 4:
            continue

        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 3.0)
        if H is None or mask is None or mask.sum() < 10:
            continue

        inliers = int(mask.sum())

        # ---------- 描画 ----------
        canvas = frame_bgr.copy()

        # 1) 対応キーポイント（入力側）を緑で描画
        matched_kp = [kp_frame[m.trainIdx] for m in good]
        cv2.drawKeypoints(canvas, matched_kp, canvas,
                          color=(0,255,0),
                          flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        # 2) テンプレート枠を射影して多角形で描画（赤）
        proj = cv2.perspectiveTransform(tmpl_corners, H)
        cv2.polylines(canvas, [np.int32(proj)], True, (0,0,255), 2, cv2.LINE_AA)

        # 3) ラベル＆インライヤ数を表示
        x, y = np.int32(proj[0,0])
        cv2.putText(canvas, f"{label}:{inliers}", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2, cv2.LINE_AA)

        annotated_imgs.append((canvas, label, inliers))
    return annotated_imgs

# --- ③ 並列保存ループ ---
executor = cf.ThreadPoolExecutor()

for frame_id, frame in enumerate(frames_from_camera()):
    anns = detect_and_annotate(frame)
    print(f"frame {frame_id}: {len(anns)} matches")

    for i, (ann_img, label, score) in enumerate(anns):
        fname = os.path.join(output_dir, f"{label}_{frame_id}_{i}_{score}.png")
        executor.submit(cv2.imwrite, fname, ann_img)

executor.shutdown(wait=True)

In [ ]:
# 閾値の設定
threshold = 200
orange=cv2.imread(orange_temp, cv2.IMREAD_GRAYSCALE)
# 二値化(閾値100を超えた画素を255にする。)
ret, img_thresh = cv2.threshold(orange, threshold, 255, cv2.THRESH_BINARY)
kernel = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)
cv2_imshow(opening)

In [ ]:
import cv2


# 画像の読み込み（グレースケールで）
img = cv2.imread(orange_temp, cv2.IMREAD_GRAYSCALE)

# ORB 特徴点検出
orb = cv2.ORB_create(nfeatures=500)
kp, des = orb.detectAndCompute(img, None)

# SIFT 特徴点検出
sift = cv2.SIFT_create()
kp, des = sift.detectAndCompute(opening, None)

# 特徴点を描画（色は緑）
img_kp = cv2.drawKeypoints(img, kp, None, color=(0,255,0),
                           flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# 表示（または保存）
cv2_imshow(img_kp)

In [ ]:
import cv2
import numpy as np

# --- 1. 画像読み込み（グレースケール） ---
img1 = cv2.imread(orange_temp, cv2.IMREAD_GRAYSCALE)  # テンプレート画像
img2 = cv2.imread(os.path.join(input_dir, 'orange_58.jpg'), cv2.IMREAD_GRAYSCALE)  # 入力画像

# --- 2. 特徴点抽出（SIFT） ---
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(opening, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# --- 3. マッチング（Brute-Force + L2距離） ---
bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.knnMatch(des1, des2, k=2)

# --- 4. Ratio Test（Lowe’s ratio）で良いマッチだけ選ぶ ---
good = []
for m, n in matches:
    if m.distance < 0.95 * n.distance:
        good.append(m)

# --- 5. マッチング結果の描画 ---
img_matches = cv2.drawMatches(
    img1, kp1,
    img2, kp2,
    good, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

# --- 6. 表示または保存 ---
cv2_imshow(img_matches)
